In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyyaml pandas scikit-learn albumentations segmentation-models-pytorch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.2 MB/s eta 0:00:00


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import pandas as pd
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np

# --- 1. Configuration (UPDATED FOR 512x512) ---
CONFIG = {
    "IMAGE_DIR": "/content/drive/MyDrive/CAF-GAN/mimic-cxr-jpg-2.0.0/files/",
    "TRAIN_CSV_PATH": "/content/drive/MyDrive/CAF-GAN/data/splits/train.csv",
    "VAL_CSV_PATH": "/content/drive/MyDrive/CAF-GAN/data/splits/val.csv",
    "OUTPUT_DIR": "/content/drive/MyDrive/CAF-GAN/outputs/cdiag_512/", # <-- New output dir
    "MODEL_NAME": "best_cdiag_512.pth",                               # <-- New model name
    "IMG_SIZE": 512,                                                 # <-- UPDATED resolution
    "BATCH_SIZE": 16,        # <-- REDUCED batch size for larger images to prevent memory errors
    "EPOCHS": 20,
    "LEARNING_RATE": 1e-4,
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    "NUM_WORKERS": 2
}
print("✅ Configuration loaded for 512x512 Cdiag training.")
print(CONFIG)

# --- 2. PyTorch Dataset (MODIFIED FOR ROBUSTNESS) ---
class MIMICCXRClassifierDataset(Dataset):
    def __init__(self, df, image_dir, img_size, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.img_size = img_size # Store image size
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        subject_id, study_id, dicom_id = str(row['subject_id']), str(row['study_id']), row['dicom_id']

        image_path = os.path.join(
            self.image_dir, f'p{subject_id[:2]}', f'p{subject_id}', f's{study_id}', f'{dicom_id}.jpg'
        )

        # Load high-resolution image
        image = Image.open(image_path).convert("RGB")

        # --- KEY CHANGE: Resize the image BEFORE converting to array and augmenting ---
        image = image.resize((self.img_size, self.img_size), Image.LANCZOS)
        image = np.array(image)
        # -----------------------------------------------------------------------------

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        label = torch.tensor(row['Pneumonia'], dtype=torch.float32)

        return image, label.unsqueeze(0)

# --- 3. Training & Validation Functions (Unchanged) ---
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    return running_loss / len(dataloader.dataset)

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss, correct_preds = 0.0, 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            preds = torch.sigmoid(outputs) > 0.5
            correct_preds += (preds == labels).sum().item()
    val_loss = running_loss / len(dataloader.dataset)
    val_acc = correct_preds / len(dataloader.dataset)
    return val_loss, val_acc

# --- 4. Main Training Execution ---
def run_training():
    DEVICE = CONFIG['DEVICE']
    os.makedirs(CONFIG['OUTPUT_DIR'], exist_ok=True)

    train_transform = A.Compose([
        A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']), # Kept as a safeguard
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=15, p=0.7),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    val_transform = A.Compose([
        A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    train_df = pd.read_csv(CONFIG['TRAIN_CSV_PATH'])
    val_df = pd.read_csv(CONFIG['VAL_CSV_PATH'])

    # --- KEY CHANGE: Pass IMG_SIZE to the Dataset constructor ---
    train_dataset = MIMICCXRClassifierDataset(train_df, CONFIG['IMAGE_DIR'], CONFIG['IMG_SIZE'], train_transform)
    val_dataset = MIMICCXRClassifierDataset(val_df, CONFIG['IMAGE_DIR'], CONFIG['IMG_SIZE'], val_transform)

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=CONFIG['NUM_WORKERS'])
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=CONFIG['NUM_WORKERS'])

    model = models.resnet50(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(model.fc.in_features, 1) # Tailor for binary classification
    model.to(DEVICE)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=CONFIG['LEARNING_RATE'])

        # Resume if checkpoint exists
    checkpoint_path = os.path.join(CONFIG['OUTPUT_DIR'], "checkpoint.pth")
    start_epoch, best_val_acc = 0, 0.0

    if os.path.exists(checkpoint_path):
        print(f"✅ Resuming from checkpoint: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        best_val_acc = checkpoint["best_val_acc"]

    print(f"\n🏋️‍♀️ Starting Cdiag training at epoch {start_epoch+1}/{CONFIG['EPOCHS']}...")

    # Training loop
    for epoch in range(start_epoch, CONFIG['EPOCHS']):
        print(f"\n--- Epoch {epoch+1}/{CONFIG['EPOCHS']} ---")
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, DEVICE)
        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc:.4f}")

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_path = os.path.join(CONFIG['OUTPUT_DIR'], CONFIG['MODEL_NAME'])
            torch.save(model.state_dict(), best_model_path)
            print(f"✨ Best model updated: {best_model_path} (Accuracy: {val_acc:.4f})")

        # Always save checkpoint
        checkpoint = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "best_val_acc": best_val_acc
        }
        torch.save(checkpoint, checkpoint_path)
        print(f"💾 Checkpoint saved at {checkpoint_path} (Epoch {epoch+1})")

    print(f"\n✅ Training complete! Best validation accuracy: {best_val_acc:.4f}")

run_training()

✅ Configuration loaded for 512x512 Cdiag training.
{'IMAGE_DIR': '/content/drive/MyDrive/CAF-GAN/mimic-cxr-jpg-2.0.0/files/', 'TRAIN_CSV_PATH': '/content/drive/MyDrive/CAF-GAN/data/splits/train.csv', 'VAL_CSV_PATH': '/content/drive/MyDrive/CAF-GAN/data/splits/val.csv', 'OUTPUT_DIR': '/content/drive/MyDrive/CAF-GAN/outputs/cdiag_512/', 'MODEL_NAME': 'best_cdiag_512.pth', 'IMG_SIZE': 512, 'BATCH_SIZE': 16, 'EPOCHS': 20, 'LEARNING_RATE': 0.0001, 'DEVICE': 'cuda', 'NUM_WORKERS': 2}
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 133MB/s]


✅ Resuming from checkpoint: /content/drive/MyDrive/CAF-GAN/outputs/cdiag_512/checkpoint.pth

🏋️‍♀️ Starting Cdiag training at epoch 15/20...

--- Epoch 15/20 ---


Validating: 100%|██████████| 19/19 [01:52<00:00,  5.94s/it]


Train Loss: 0.2936 | Val Loss: 0.9298 | Val Accuracy: 0.6233
💾 Checkpoint saved at /content/drive/MyDrive/CAF-GAN/outputs/cdiag_512/checkpoint.pth (Epoch 15)

--- Epoch 16/20 ---


...

Train Loss: 0.2161 | Val Loss: 0.7944 | Val Accuracy: 0.7033
💾 Checkpoint saved at /content/drive/MyDrive/CAF-GAN/outputs/cdiag_512/checkpoint.pth (Epoch 18)

--- Epoch 19/20 ---


Validating: 100%|██████████| 19/19 [00:41<00:00,  2.16s/it]


Train Loss: 0.1822 | Val Loss: 1.0991 | Val Accuracy: 0.6400
💾 Checkpoint saved at /content/drive/MyDrive/CAF-GAN/outputs/cdiag_512/checkpoint.pth (Epoch 19)

--- Epoch 20/20 ---


Validating: 100%|██████████| 19/19 [00:41<00:00,  2.18s/it]


Train Loss: 0.2119 | Val Loss: 0.9077 | Val Accuracy: 0.6500
💾 Checkpoint saved at /content/drive/MyDrive/CAF-GAN/outputs/cdiag_512/checkpoint.pth (Epoch 20)

✅ Training complete! Best validation accuracy: 0.7567
